Benefits of GPU

Devoting more transistors to data processing, for example,
floating-point computations, is beneficial for highly parallel computations;
the GPU can hide memory access latencies with computation, instead of relying on large data caches and comples flow control to avoid long memory access latencies, both of which are expensive in term of transistors.

in general , an application has a mix of parallel parts and sequential parts, sy system are designed with a mix of GPUs and CPUs in order to maximize overall performance.
Applications with a  high degree of parallelism can exploit this massively parallel nature of the GPU to achieve higher performance than on the CPU

.



A Scalable Programming Model

The advent of multicore CPUs and manycore GPUs means that mainstream processor chips are now parallel systems.The challenge is to develop application software that transparently scales it parallelism to leverage the increasing number of processor cores, much as 3D graphic applications transparently scale their parallelism to manycore GPUs with widely varying numbers of cores.

The CUDA parallel programming model is designed to overcome this challenge while maintaining a low learning curve for programmers familiar with standard programming languages such a sC.

At its core are three key abstractions - a hierarchy of thread groups, shared memories, and barrier synchronization-
that are simply exposed to the programmer as a minimal set of language extensions.

These abstractions provide fine-grained data parallelism and thread parallelism, nested within coarse-grained data parallelism and task parallelism.
They guide the programmer to partition the problem into coarse sub-problems that can be solved independently in parallel by blocks of threads, and each subproblem into finer pieces that can be solved cooperatively in parallel by all threads within the block.

This decomposition preserves language expressivity by allowing threads to cooperate when solving each sub-problem, and at the same time enables automatic scalability.
Indeed, each block of threads can be scheduled on any of the available multiprocessors within a GPU, in any order, concurrently or sequentially,
so that a compiled CUDA program cna excute on any number of multiprocessors as illustrated by PIC 3,
and only the runtime system needs to know the physical multiprocessor count.

This scalable programming model allows the GPU architecture to span a wide market range by simply scaling the number of multiprocessors and memory partitions: from the high-performance enthusiast Gpu, adn professional Quadro &Tesla computing products to a variety of inexpensive, mainstream Geforce GPU.

A GPU is built around an array of Streaming Multiprocessors(SMs).
A multithreaded program is partitioned into blocks of threads that execute independently from each other, so that a GPU with more multiprocessors will automatically execute the program in less time than a GPU with fewer multiprocessors.






Cuda programming Model

This chapter introduces the main concepts behind the CUDA programming model by outlining how they are exposed in C++.

An extensive description of CUDA C++ is given in Programming Interface.

2.1 Kernels 

CUDA C++ extends C== by allowing the programmer to define C++ functions, called kernels, that, when called, are executed N times in parallel by N different CUDA threas, as opposed to only once like regular C++ functions.

A kernel is defined using the __global__ declaration specifier and the number of CUDA threads that execute that kernel for a given kernel call is specified using a new <<<>>> execution contiguration syntax.

Each thread that executes the kernel is given a unique *thread ID* that is **accessible within the kernel** through built-in variables.



As an illustration, the following sample code, using the built-in variable *threadIdx*, adds two vectors A and B of size N and stores the result into vector C:

In [ ]:
// Kernel definition
__global__ void VecAdd(float* A, float* B, float* C)
{
    int i = threadIdx.x;
    C[i] = A[i] + B[i];
}

int main()
{
    ...
    // Kernel invocation with N threads
    VecAdd<<<1, N>>>(A, B, C);
    ...
}

2.2 Thread Hierarchy 

For convenience ,*threadIdx* is a 3-component vector,s othat threads can be identified using a one-dimensional, two-dimensional, or three- dimensional thread index, forming a one=dimensional ,two-dimensional ,or three-dimensional block of threads, called a **thread block**.
This provides a natual way to invoke computation across the elements in a domain such as a vector, matrix, or volume.

The index of a thread and its thread ID relate to each other in a straight forward way: 

For a one-dimensional block ,they are the same;

for a two-dimensional block of size *Dx,Dy*,the thread ID of a thread of index
(x,y) is *(x+y Dx*);

 the thread ID of a thread of index (x, y, z) is *(x + y Dx + z Dx Dy)*.





In [ ]:
//As an example, the following code adds two matrices A and B of size NxN and stores the result into matrix C:

In [ ]:
// Kernel definition
__global__ void MatAdd(float A[N][N], float B[N][N],
                       float C[N][N])
{
    int i = threadIdx.x;
    int j = threadIdx.y;
    C[i][j] = A[i][j] + B[i][j];
}

int main()
{
    ...
    // Kernel invocation with one block of N * N * 1 threads
    int numBlocks = 1;
    dim3 threadsPerBlock(N, N);
    MatAdd<<<numBlocks, threadsPerBlock>>>(A, B, C);
    ...
}

In [ ]:
Block , contains threads per block.

total_threas = Block* threads per block

There is a limit to the number of threads per block, 

 since **all threads of a block** are expected to reside on the same *streaming multiprocessor core* 
and must share the limited memory resources of that core. 

On current GPUs, a thread block may contain up to 1024 threads.


However, a *kernel* can be executed by multiple *equally-shaped thread blocks*, so that the total number of threads is equal to the number of threads per block times the number of blocks.

Blocks are organized into a one-dimensional, two-dimensional, or three-dimensional grid of thread blocks as illustrated by Figure 4.

the number of blocks(contain threads) in a grid is usually dictated by the size of the data being processed, which typically exceeds the number of processors in the system.



*The number of threads per block* and *the number of blocks per grid *specified in the <<<...>>> syntax can be of type int or dim3. Two-dimensional blocks or grids can be specified as in the example above.

Each *block* within the *grid* can be identified by a one-dimensional, two-dimensional, or three-dimensional unique index accessible within the kernel through the built-in **blockIdx** variable. The dimension of the thread block is accessible within the kernel through the built-in **blockDim** variable.

Extending the previous MatAdd() example to handle multiple blocks, the code becomes as follows.

In [ ]:
// Kernel definition
__global__ void MatAdd(float A[N][N], float B[N][N],
float C[N][N])
{
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;
    if (i < N && j < N)
        C[i][j] = A[i][j] + B[i][j];
}

int main()
{
    ...
    // Kernel invocation
    dim3 threadsPerBlock(16, 16);
    dim3 numBlocks(N / threadsPerBlock.x, N / threadsPerBlock.y);
    MatAdd<<<numBlocks, threadsPerBlock>>>(A, B, C);
    ...
}

A thread block size of 16x16 (256 threads), although arbitrary in this case, is a common choice. 



Thread blocks are required to execute independently: It must be possible to execute them in any order, inparallel or in series.
This independence requirement allows thread blocks to be scheduled in any order across any number of cores as illyustrated , enabling programmers to write code that scales with the number of cores.



Threads within a block can cooperate by sharing data through some shared memory and by synchronizing their execution to coordinate memory accesses.

More precisely, one can specify synchronization points in the kernel by calling the *__syncthreads()* intrinsic function;

//block 内同步锁
*__syncthreads()* acts as a barrier at which all threads in the block must wait before any is allowed to proceed.

Shared memory give an example of using shared memory.

In addition to __syncthreads(), the Cooperative Groups API provides a rich set of thread-synchronization primitives.

For efficient cooperation, the shared memory is expected to be a low-latency memory near each processor core( much like an L1 cache) and __syncthreads() is expected to be lightweeight.





A thread block cluster can be enabled in a kernel either using a compiler time kernel attribute using __cluster_dims__(X,Y,Z) or using the CUDA kernel launch API cudaLaunchKernelEx. The example below shows how to launch a cluster using compiler time kernel attribute. The cluster size using kernel attribute is fixed at compile time and then the kernel can be launched using the classical <<< , >>>. If a kernel uses compile-time cluster size, the cluster size *cannot* be modified when launching the kernel.

In [ ]:
// Kernel definition
// Compile time cluster size 2 in X-dimension and 1 in Y and Z dimension
__global__ void __cluster_dims__(2, 1, 1) cluster_kernel(float *input, float* output)
{

}

int main()
{
    float *input, *output;
    // Kernel invocation with compile time cluster size
    dim3 threadsPerBlock(16, 16);
    dim3 numBlocks(N / threadsPerBlock.x, N / threadsPerBlock.y);

    // The grid dimension is not affected by cluster launch, and is still enumerated
    // using number of blocks.
    // The grid dimension must be a multiple of cluster size.
    cluster_kernel<<<numBlocks, threadsPerBlock>>>(input, output);
}

A thread block cluster size can also be set at runtime and the kernel can be launched using the CUDA kernel launch API **cudaLaunchKernelEx**. The code example below shows how to launch a cluster kernel using the extensible API.

In [ ]:
// Kernel definition
// No compile time attribute attached to the kernel
__global__ void cluster_kernel(float *input, float* output)
{

}

int main()
{
    float *input, *output;
    dim3 threadsPerBlock(16, 16);
    dim3 numBlocks(N / threadsPerBlock.x, N / threadsPerBlock.y);

    // Kernel invocation with runtime cluster size
    {
        cudaLaunchConfig_t config = {0};
        // The grid dimension is not affected by cluster launch, and is still enumerated
        // using number of blocks.
        // The grid dimension should be a multiple of cluster size.
        config.gridDim = numBlocks;
        config.blockDim = threadsPerBlock;

        cudaLaunchAttribute attribute[1];
        attribute[0].id = cudaLaunchAttributeClusterDimension;
        attribute[0].val.clusterDim.x = 2; // Cluster size in X-dimension
        attribute[0].val.clusterDim.y = 1;
        attribute[0].val.clusterDim.z = 1;
        config.attrs = attribute;
        config.numAttrs = 1;

        cudaLaunchKernelEx(&config, cluster_kernel, input, output);
    }
}

2.3. Memory Hierarchy

CUDA threads may access data from multiple memory spaces during their execution

Each thread has private local memory. 


Each thread block has shared memory visible to all threads of the block and with the same lifetime as the **block**.

*Thread blocks* in a thread block **cluster** can perform read, write, and atomics operations on **each other’s shared memory**.


here are also two additional read-only memory spaces accessible by all threads: 

 the constant 
 and texture memory spaces. 


The global, constant, and texture memory spaces are optimized for different memory usages 

 Texture memory also offers different addressing modes, as well as data filtering, for some specific data formats 

 The global,
  constant,
   and texture memory spaces are persistent **across kernel launches** by the same application.


2.4. Heterogeneous Programming

the CUDA programming model assumes that the CUDA threads execute on a physically separate device that operates as a coprocessor to the host running the C++ program. This is the case, for example,

when the kernels execute on a GPU and the rest of the C++ program executes on a CPU.

The CUDA programming model also assumes 
that both the host and the device maintain their own separate memory spaces in DRAM, 
referred to as *host memory* and *device memory*, respectively. 

Therefore, a program manages the global, constant, and texture memory spaces visible to kernels through 
calls to the CUDA runtime

 This includes device memory *allocation* and *deallocation* as well as *data transfer* between host and device memory.

Unified Memory provides managed memory to bridge the host and device memory spaces. 

Managed memory is accessible from all CPUs and GPUs in the system 
as a single, coherent memory image with a common address space. 

 This capability enables oversubscription of device memory and can greatly simplify the task of porting applications by eliminating the need to explicitly mirror data on host and device. 

2.5. Asynchronous SIMT Programming Model

In the CUDA programming model 
a thread is the lowest level of abstraction for doing a computation or a memory operation. 

Starting with devices based on the *NVIDIA Ampere* GPU architecture, the CUDA programming model provides acceleration to memory operations via the asynchronous programming model. 

The asynchronous programming model 
defines the behavior of *asynchronous* operations with respect to **CUDA threads**.

The asynchronous programming model defines the behavior of *Asynchronous Barrier* for *synchronization* between CUDA threads. 

The model also explains and defines how **cuda::memcpy_async** can be used to move data asynchronously from global memory while **computing in the GPU**.


2.5.1. Asynchronous Operations

